In [1]:
from tqdm.notebook import tqdm_notebook as tqdm

import numpy as np
import random
import shutil
import json
import glob
import cv2
import os

In [2]:
def create_flip_dataset(dir_name):
        all_image = ( i for i in glob.iglob("Dental_Data/PBL/%s/*.png" % dir_name) )

        destination_dir = "Dental_Data/PBL/%s_Flip" % dir_name
        json_file = "Dental_Data/PBL/%s/mapping.json" % dir_name
        if not os.path.isdir(destination_dir): os.makedirs(destination_dir)
        
        mapping_data = json.load(open("Dental_Data/PBL/%s/mapping.json" %dir_name, "r"))
        filter_mapping_data_Flip  = { path.replace(dir_name, "%s_Flip" % dir_name): state for path, state in mapping_data.items()  }
        json.dump(filter_mapping_data_Flip, open(json_file.replace( dir_name , "%s_Flip" % dir_name ), 'w'), indent=4)
#         shutil.copy(json_file, json_file.replace( dir_name , "%s_Flip" % dir_name ))

        for image_path in tqdm(all_image):
                image = cv2.imread(image_path, 0)
                image = cv2.flip(image, 1)
                image_path = image_path.replace(dir_name, "%s_Flip" % dir_name)
                cv2.imwrite(image_path, image)
                
def create_shear_dataset(dir_name):
        all_image = ( i for i in glob.iglob("Dental_Data/PBL/%s/*.png" % dir_name) )

        destination_dir = "Dental_Data/PBL/%s_Shear" % dir_name
        json_file = "Dental_Data/PBL/%s/mapping.json" % dir_name
        if not os.path.isdir(destination_dir): os.makedirs(destination_dir)
        
        mapping_data = json.load(open("Dental_Data/PBL/%s/mapping.json" %dir_name, "r"))
        filter_mapping_data_Flip  = { path.replace(dir_name, "%s_Shear" % dir_name): state for path, state in mapping_data.items()  }
        json.dump(filter_mapping_data_Flip, open(json_file.replace( dir_name , "%s_Shear" % dir_name ), 'w'), indent=4)

        shear_range  = ( -0.1 , 0.1)
        
        for image_path in tqdm(all_image):
                shear_factor = random.uniform(*shear_range)
                image = cv2.imread(image_path, 0)
                H, W = image.shape
                
                corner = np.float32([ 
                      [0, 0],
                      [H, 0],
                      [H, W],
                      [0, W]
                   ])
        
                affine = np.float32([ 
                            [1, shear_factor],
                            [0, 1],
                         ])
        
                new_corner = np.dot(corner, affine)
                negative = new_corner[new_corner<0]
                
                if len(negative) == 0: M2 = np.float32([[1, shear_factor, 0], [0, 1, 0]])
                else: 
                        negative = min(negative)
                        M2 = np.float32([[1, shear_factor, -negative], [0, 1, 0]])

                nW = image.shape[1] + abs(shear_factor*image.shape[0])
                
                image = cv2.warpAffine(image, M2, (int(nW), H))
                image_path = image_path.replace(dir_name, "%s_Shear" % dir_name)
                cv2.imwrite(image_path, image)
                
        
                
# flip_dataset = [ "10_interdental_20200901_Max_4_HV", "10_interdental_clahe_20200901_Max_4_HV",
#                  "10_interdental_20200901_Max_4_stage", "10_interdental_clahe_20200901_Max_4_stage"
#                ]



flip_dataset = [ "10_interdental_20200901_Max_4", "10_interdental_clahe_20200901_Max_4" ]
# #                  "5_20200810" , "5_clahe_20200810" , "5_interdental_20200810" , "5_interdental_clahe_20200810" ]

for dataset in flip_dataset:
#         create_flip_dataset(dataset)
        create_shear_dataset(dataset)

In [3]:
# import matplotlib.pyplot as plt
# image = cv2.imread("Dental_Data/PBL/10_clahe_20200901/16-31-08-628_000408 102419 x_NN_191024_151623_BE78A8_6_-10.png", 0)

# rows,cols = image.shape  
# # Shear
# pts1 = np.float32([[5,5],[20,5],[5,20]])
# shear_range = 5
# pt1 = 5  + shear_range*np.random.uniform()-shear_range/2
# pt2 = 20 + shear_range*np.random.uniform()-shear_range/2
# pts2 = np.float32([[pt1,5],[pt2,pt1],[5,pt2]])
# shear_M = cv2.getAffineTransform(pts1,pts2)
# img = cv2.warpAffine(image,shear_M,(cols,rows))

# plt.subplot(121)
# plt.imshow(image, cmap='gray')
# plt.axis('off')

# plt.subplot(122)
# plt.imshow(img, cmap='gray')
# plt.axis('off')
# plt.show()